# Data PreProcessing

In [23]:
import sys
sys.path.append('/Users/mac/Desktop/Desktop_Files/Projects/Zomato Restaurant Rating Prediction')


In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from utility.utils import analyze_corr, convert_cat

In [16]:
## laoding the dataset (EDA_final_cleaned.pkl)
import joblib
df = joblib.load("../artifacts/EDA_final_cleaned.pkl")

In [17]:
df.head(2)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost,reviews_list,menu_item,type,city,rating_group,cuisines_list
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,1,1,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3 to 4.2,"[North Indian, Mughlai, Chinese]"
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,1,0,4.1,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3 to 4.2,"[Chinese, North Indian, Thai]"


#### Categorical Conversion

In [18]:
## dropping cuisine column as it is not categorical->
df = df.drop(columns=['cuisines'], axis=1)

Note-> The dtype will still be int because we are also puttin integer codes for those categoricla columns 
- u can check the utils.py file having convert_cat fucntion with .cat.codes feature in it.

In [19]:
## converting  columns to category dtype
col_list = ['online_order', 'book_table', 'location', 'rest_type', 'type', 'city']
df = convert_cat(df, col_list)


In [20]:
df[col_list].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41665 entries, 0 to 41664
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   online_order  41665 non-null  int8 
 1   book_table    41665 non-null  int8 
 2   location      41665 non-null  int8 
 3   rest_type     41665 non-null  int8 
 4   type          41665 non-null  int8 
 5   city          41665 non-null  int8 
dtypes: int8(6)
memory usage: 244.3 KB


In [80]:
df['approx_cost'].sample(5)

22601    350
38660    300
19545    350
23192    400
22445    200
Name: approx_cost, dtype: int64

### Outliers Detection->

In [ ]:
## Function to check for outliers using IQR method
def check_outliers(df, col):
    s = df[col]   ## selecting the series (column)

    # calculatin both quartiles
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1

    # bounds
    lower_bound, upper_bound = q1 - 1.5 * iqr, q3 + 1.5 * iqr

    # Vectorized checks for outliers
    has_lower = (s < lower_bound).any()
    has_upper = (s > upper_bound).any()

    # Print summary
    print(f"IQR: {iqr}")
    print(f"Lower cutoff: {lower_bound}")
    print(f"Upper cutoff: {upper_bound}\n")

    print("Lower Side:", "Outliers present 🚩" if has_lower else "No outliers ✅")
    print("Upper Side:", "Outliers present 🚩" if has_upper else "No outliers ✅")




In [88]:
check_outliers(df, 'approx_cost')

IQR: 300.0
Lower cutoff: -250.0
Upper cutoff: 950.0

Lower Side: No outliers ✅
Upper Side: No outliers ✅


- Fortunately, our data has no outliers for "approx_cost" column

Let's check for all other columns->

In [89]:
df.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'approx_cost',
       'reviews_list', 'menu_item', 'type', 'city', 'rating_group',
       'cuisines_list'],
      dtype='object')

In [ ]:
# online_order 
check_outliers(df, 'online_order' ) # obv no outliers bcz categorical

IQR: 1.0
Lower cutoff: -1.5
Upper cutoff: 2.5

Lower Side: No outliers ✅
Upper Side: No outliers ✅


In [90]:
check_outliers(df, 'rate' )

IQR: 0.6000000000000001
Lower cutoff: 2.5
Upper cutoff: 4.9

Lower Side: Outliers present 🚩
Upper Side: No outliers ✅


In [93]:
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        print(f"Checking outliers for column: {col}")
        check_outliers(df, col)
        print("-" * 40)

Checking outliers for column: rate
IQR: 0.6000000000000001
Lower cutoff: 2.5
Upper cutoff: 4.9

Lower Side: Outliers present 🚩
Upper Side: No outliers ✅
----------------------------------------
Checking outliers for column: votes
IQR: 255.0
Lower cutoff: -361.5
Upper cutoff: 658.5

Lower Side: No outliers ✅
Upper Side: Outliers present 🚩
----------------------------------------
Checking outliers for column: approx_cost
IQR: 300.0
Lower cutoff: -250.0
Upper cutoff: 950.0

Lower Side: No outliers ✅
Upper Side: No outliers ✅
----------------------------------------
